In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from utils import clean_target
from categorical_ordinal import get_categorical_ordinal_columns
from categorical_nominal import get_categorical_nominal_columns
from columns_transformers import ColumnSelector
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

<font color="orange"> <b>Grupos de features:</b></font>

- **Categorial Ordinal:**
    - TP_ (17-4-3 = 10)
    - Questions: ```["Q001", "Q002", "Q003", "Q004", "Q005","Q006", "Q007", "Q008", "Q009", "Q010", "Q011", "Q012", "Q013", "Q014", "Q015", "Q016", "Q017", "Q019", "Q022", "Q024"]``` (20)
                     
- **Categorial Nominal:**
    - IN_ : All Binary (52) 
    - TP_ : ```["TP_SEXO", "TP_ESTADO_CIVIL", "TP_COR_RACA", "TP_NACIONALIDADE"]``` (4)
    - SG_ : (4-1 = 3)
    - Questions: ```["Q018", "Q020", "Q021", "Q023", "Q025"]``` (5)
    
- **Numerical:**
    - NU_IDADE (1)

- Droped:
    - Identificator: ```[NU_INSCRICAO]``` (1)
    - More than 40% missing: ```['CO_ESCOLA', 'NO_MUNICIPIO_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC']``` (4)
    - NO_M: (To many categories): ```['NO_MUNICIPIO_RESIDENCIA', 'NO_MUNICIPIO_NASCIMENTO', 'NO_MUNICIPIO_PROVA']``` (3)
    - NU_NOTA: Targets variables (5)

In [2]:
train_df = pd.read_parquet("data/train.parquet")
clean_target(train_df)
#test= pd.read_parquet("data/test.parquet")

In [3]:
categorical_ordinal_columns = get_categorical_ordinal_columns(train_df)
qtd_categorical_ordinal_columns=len(categorical_ordinal_columns)
print(f"Number of categorial ordinal features: {qtd_categorical_ordinal_columns}")

Number of categorial ordinal features: 30


In [4]:
categorical_nominal_columns = get_categorical_nominal_columns(train_df)
qtd_categorical_nominal_columns = len(categorical_nominal_columns)
print(f"Number of categorial nominal features: {qtd_categorical_nominal_columns}")

Number of categorial nominal features: 64


In [5]:
drop_columns = ["NU_INSCRICAO", 
                "CO_ESCOLA", 
                "NO_MUNICIPIO_ESC", 
                "SG_UF_ESC", 
                "TP_DEPENDENCIA_ADM_ESC", 
                "TP_LOCALIZACAO_ESC", 
                "TP_SIT_FUNC_ESC", 
                "NO_MUNICIPIO_RESIDENCIA", 
                "NO_MUNICIPIO_NASCIMENTO", 
                "NO_MUNICIPIO_PROVA"]
qtd_drop_columns = len(drop_columns)
print(f"Number of columns dropped: {qtd_drop_columns}")

Number of columns dropped: 10


In [6]:
target_columns = train_df.filter(regex="NU_NOTA").columns.tolist()
qtd_target_columns = len(target_columns)
print(f"Number of targets: {qtd_target_columns}")

Number of targets: 5


In [7]:
numerical_columns = ["NU_IDADE"]
qtd_numerical_columns = len(numerical_columns)
print(f"Number of targets: {qtd_numerical_columns}")

Number of targets: 1


In [8]:
target_columns = train_df.filter(regex="NU_NOTA").columns.tolist()
qtd_target_columns = len(target_columns)
print(f"Number of targets: {qtd_target_columns}")

Number of targets: 5


In [9]:
all_columns = drop_columns + categorical_nominal_columns + categorical_ordinal_columns + numerical_columns + target_columns
qtd_total = qtd_drop_columns + qtd_categorical_nominal_columns + qtd_categorical_ordinal_columns + qtd_numerical_columns + qtd_target_columns
print(f"Total columns: {qtd_total}")

Total columns: 110


## **Create Pipeline**

In [16]:
"""
Variáveis categóricas com dados ordinais que tem dados faltantes:
- TP_ENSINO: Suposto que NaN representa a categoria faltante descrita nos metadados.
- TP_STATUS_REDACAO: Mapeado para outra classe (Faltou na prova)
"""
categorical_ordinal_pipe = Pipeline([
    ('selector', ColumnSelector(categorical_ordinal_columns)),
    ('imputer', SimpleImputer(missing_values=np.nan, 
                              strategy='constant', 
                              fill_value=0)),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

"""
Variáveis categóricas com dados ordinais que tem dados faltantes:
- SG_UF_NASCIMENTO: Mapeado para uma nova categoria
"""
categorical_nominal_pipe = Pipeline([
    ('selector', ColumnSelector(categorical_nominal_columns)),
    ('imputer', SimpleImputer(missing_values=np.nan, 
                              strategy='constant', 
                              fill_value="missing")),
    ('encoder', OneHotEncoder(drop="first", handle_unknown='ignore'))
])

numerical_pipe = Pipeline([
    ('selector', ColumnSelector(numerical_columns)),
    ('imputer', SimpleImputer(missing_values=np.nan, 
                              strategy='constant', 
                              fill_value=0)),
    ('scaler', MinMaxScaler())
])


preprocessor = FeatureUnion([
    ('categorical_ordinal', categorical_ordinal_pipe),
    ('categorical_nominal', categorical_nominal_pipe),
    ('numerical', numerical_pipe)
])

kwargs_regressor = {"n_estimators":100,
                    "max_depth":20,
                    "n_jobs":-1,
                    "verbose":2}

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(**kwargs_regressor))
])

In [17]:
def presence_filter(df, key):
    cond = df.filter(regex=f"PRESENCA_{key}|STATUS_{key}").iloc[:,0] == 1
    return df.loc[cond, :], df.loc[~cond, :]


y_structure = {"CN":[], 
               "CH":[],
               "LC":[],
               "MT":[],
               "REDACAO":[]}

In [18]:
from joblib import dump
n_samples = 20000
sample = train_df.sample(n_samples)
    
for key, ys in tqdm(y_structure.items()):
    filtered_sample, missed_exam = presence_filter(sample, key)
    X = filtered_sample.drop(columns=target_columns+drop_columns)
    y = filtered_sample.filter(regex=f"NU_NOTA_{key}")
    
    X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.1, random_state=42)
    pipe.fit(X_train, y_train)
    dump(pipe, f"models/model_{key}.joblib") 
    y_train_hat = pipe.predict(X_train)
    y_test_hat = pipe.predict(X_test)
    ys.extend([y_train, y_test, y_train_hat, y_test_hat])

  0%|                                                                                                                                                                                    | 0/5 [00:00<?, ?it/s]/home/rocabrera/.pyenv/versions/3.9.9/envs/venv_playground/lib/python3.9/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100building tree 5 of 100

building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.8s


building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   19.4s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished
/home/rocabrera/.pyenv/versions/3.9.9/envs/venv_playground/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:170: UserWarning: Found unknown categories in columns [3, 14, 29] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
 20%|██████████████████████████████████▍                                                                                                                                         | 1/5 [00:20<01:21, 20.49s/it]/home/rocabrera/.pyenv/

...

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   22.8s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished
/home/rocabrera/.pyenv/versions/3.9.9/envs/venv_playground/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:170: UserWarning: Found unknown categories in columns [3, 14, 29] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                  | 4/5 [01:27<00:22, 22.33s/it]/home/rocabrera/.pyenv/

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.0s


building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   20.7s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:49<00:00, 21.83s/it]


In [19]:
mean_train = []
mean_test = []
for key, ys in tqdm(y_structure.items()):
    train_error = mean_squared_error(ys[0], ys[2], squared=False)
    mean_train.append(train_error)
    test_error = mean_squared_error(ys[1], ys[3], squared=False)
    mean_test.append(test_error)
    print(key)
    print(f"Train: {train_error}")
    print(f"Test: {test_error}\n")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 411.99it/s]

CN
Train: 30.481697652420085
Test: 66.09322869143541

CH
Train: 35.56581610673091
Test: 70.7716822879178

LC
Train: 26.736370294377817
Test: 54.45754639381666

MT
Train: 40.6420401639734
Test: 90.4254604343699

REDACAO
Train: 69.39820823679386
Test: 135.13763215413306



In [14]:
np.mean(mean_train)

31.36771236248062

In [15]:
np.mean(mean_test)

83.8079209916267